In [1]:
%load_ext autoreload
%autoreload 2

In [12]:
import os
import shutil
import zipfile
import pprint
from pathlib import Path
import pickle
import sys
from functools import partialmethod
import time

import tqdm
tqdm.__init__ = partialmethod(tqdm.__init__, disable=True) # silence tqdm
from ipywidgets import interact, interactive, fixed, interact_manual,Button, HBox, VBox
from ipywidgets import Checkbox, VBox, HBox, Text, Dropdown, Image, IntSlider
from IPython.display import Image
import ipywidgets as widgets
import matplotlib.pyplot as plt
import pandas as pd

from nlp_suite import data_preprocessing
from basic_info_widget import BasicInfoWidget 
from stylized_chatbot_widget import StylizedChatbotWidget
from cluster_widget import ClusterWidget
from senti_analysis_widget import SentiAnalysisWidget
from nlp_suite.clustering.utils import plot_3d_clusters
from nlp_suite.clustering.utils import plot_3d_clusters, classify_text
from nlp_suite.text_analysis.text_analysis_kevin import *

In [23]:
class App:
    
    def __init__(self):
        # Add Logo
        logo_file = open("./Graphics/main_logo.png", "rb")
        logo_widget = widgets.Image(value=logo_file.read(),format='png',width=150,height=150)
        # Add Title
        title_widget = widgets.HTML('<p style="font-size:50px">&nbsp;&nbsp;<b>Chat Log Analysis Suite</b></p>')
        # Instruction text UI
        instruction_text_widget = widgets.HTML('<p style="font-size:18px"><b>To begin, upload a zipped folder containing chat log file(s), and specify a user. </br>Then, click the green button to start the data analysis.</b></p>')
        blank_line_widget = widgets.HTML('<br>')
        
        # Upload input chat log UI
        self.uploader_widget = widgets.FileUpload(description="Upload Chat", accept='.zip', multiple=False)
        self.user_name_widget = widgets.Text(description='User Name:', disabled=False)
        user_input_widgets = HBox([self.uploader_widget, self.user_name_widget])
        
        # submit chat log UI
        process_data_button_widget = widgets.Button(description='Process Chat Data', disabled=False, button_style='success')
        process_data_button_widget.on_click(self.begin_analysis)
        self.status_text_widget = widgets.HTML('')
        submit_info_widgets = HBox([process_data_button_widget, self.status_text_widget])
        
        # SHOW TEXT ANALYSIS FOR CLUSTERS
        pkl_data = pickle.load(open('./nlp_suite/clustering/cluster_data.pkl', 'rb'))
        labels, pca = pkl_data['labels'], pkl_data['pca']
        self.fig = plot_3d_clusters(pca, labels, max_points=50000)
        self.cluster_dd = Dropdown(
                options=[(f'Cluster {i}', i) for i in range(max(labels)+1)],
                value=0,
                description='Cluster:',
            )
        def on_click(trace, points, state):
            self.cluster_dd.value = int(trace.marker.color[points.point_inds[0]])
        self.fig.data[0].on_click(on_click)

        # analysis tabs UI
        self.basic_info_widget = BasicInfoWidget()
        self.stylized_chatbot_widget = StylizedChatbotWidget()
        self.cluster_widget = ClusterWidget()
        self.sentiment_analysis_widget = SentiAnalysisWidget()
        analysis_tabs_child_widgets = [self.basic_info_widget.get_widget(),
                                       self.cluster_widget.get_widget(),
                                       self.sentiment_analysis_widget.get_widget(),
                                       self.stylized_chatbot_widget.get_widget()]
        analysis_tabs_children_names = ["Basic User Info", "Patterns & Clusters", "Sentiment", "Chatbot"]
        self.analysis_tabs_widget = widgets.Tab(children = analysis_tabs_child_widgets)
        for i in range(len(analysis_tabs_child_widgets)):
            self.analysis_tabs_widget.set_title(i, analysis_tabs_children_names[i])
        
        # final container for all UI
        self.container = VBox([
            HBox([
                logo_widget,
                VBox([
                    blank_line_widget,
                    blank_line_widget,
                    title_widget
                ])
            ]),
            instruction_text_widget,
            blank_line_widget,
            user_input_widgets,
            submit_info_widgets,
            blank_line_widget,
            self.analysis_tabs_widget
        ])
    
    
    # resets uploader and user name widgets
    def reset_user_input_widgets(self):
        self.uploader_widget.value.clear()
        self.uploader_widget._counter = 0
        self.user_name_widget.value = ""
            
    
    # this is run when the begin button is pressed; preprocesses the data, populates user_info, and sets up the analysis tabs
    def begin_analysis(self, button_instance):
        self.status_text_widget.value = '<p style="font-size:18px;color:orange;"><b>Processing...</b></p>'
        time.sleep(1)
        user_info = self.process_user_info()
        if user_info == {}:
            # user info loading failed
            self.status_text_widget.value = '<p style="font-size:18px;color:red;"><b>Error. Please ensure the specified name is a valid user in the uploaded chat logs.</b></p>'
            self.reset_user_input_widgets()
        else:
            # user info loading succeeded; init all analysis tab widgets
            self.reset_user_input_widgets()
            self.basic_info_widget.init_widget_data(user_info)
            self.cluster_widget.data_been_processed(user_info)
            self.sentiment_analysis_widget.init_widget_data(user_info)
            self.stylized_chatbot_widget.init_widget_data(user_info)
            self.status_text_widget.value = '<p style="font-size:18px;color:green;"><b>Finished processing user \"{}\".</b></p>'.format(user_info["user_name"])
    
    
    # preprocesses data from uploader and user_name widgets, into a user_info dict
    def process_user_info(self):
        user_name = self.user_name_widget.value
        
        if user_name == "" or self.uploader_widget.value == {}:
            return {}
        
        else:
            temp_files_dir = "./dashboard_temp_files"
            cached_user_data_dir = os.path.join("./cached_user_data", user_name)
            user_messages_path = os.path.join(cached_user_data_dir, "user_messages.p")
            
            # if cache of user messages does not exist, create it
            if not os.path.exists(user_messages_path):
                # prepping an empty temp folder to hold uploaded data
                if os.path.exists(temp_files_dir):
                    shutil.rmtree(temp_files_dir)
                os.makedirs(temp_files_dir)
                # saving zip file from uploader widget into temp folder
                uploaded_file_path = os.path.join(temp_files_dir, "saved-output.zip")
                with open(uploaded_file_path, "wb") as f:
                    content = (self.uploader_widget.value[list(self.uploader_widget.value.keys())[0]])["content"]
                    f.write(content)
                # unzipping
                with zipfile.ZipFile(uploaded_file_path, 'r') as zip_ref:
                    zip_ref.extractall(temp_files_dir)
                os.remove(uploaded_file_path)
                # extracting user's messages from text the unzipped files
                discord_log_paths = [str(path) for path in list(Path(temp_files_dir).rglob('*.txt'))]
                channel_messages, _ = data_preprocessing.process_discord_data(discord_log_paths, 3)
                # saving user messages in user cache dir, and returning user_info dict
                if user_name in channel_messages:
                    os.makedirs(cached_user_data_dir, exist_ok=True)
                    pickle.dump(channel_messages[user_name], open(user_messages_path, "wb"))
                else:
                    return {}
                
            user_messages = pickle.load(open(user_messages_path, "rb"))
            return {"user_name": user_name, "user_messages": user_messages}


In [24]:
app = App()
display(app.container)